# Create the project structure

This is a helper notebook to create the folder structure necessary for HA1.
Start looking at `HA1.ipynb` and revisit this notebook when needed.

This should be run from the same folder where the `dogs-vs-cats.zip` file you downloaded from Kaggle is.

In [ ]:
# For dealing with files we use the built-in python module `Path`
# It provides a nice abstraction of the file system, compared to working with strings only.
# It also makes your code more portable, i.e. easier to share with someone using another operating system.
from pathlib import Path
# Some file system operation are not covered by 'Path' and we use 'shutil' for that
import shutil

# Regular expressions are used to find patterns in strings
import re

# For splitting the data
from sklearn.model_selection import train_test_split

In [ ]:
# NOTE: This script assumes that you have the `dogs-vs-cats.zip` in the same directory as this notebook
zip_file = Path.cwd() / "dogs-vs-cats.zip"
if not zip_file.exists():
    raise FileNotFoundError("Download and place `{}` in the current directory (`{}`)"
                            .format(zip_file.name, Path.cwd()))

# This is a list of all the directories and files this notebook will produce.
# If you have run this before, we will delete them and start over from `dogs-vs-cats.zip`
# Notice how we use the `map` function to conveniently run `Path(<filename>)` on all strings in our list,
# to turn them in portable filepaths.
pre_existing_items = map(Path, ["test1.zip",
                     "test",
                     "val",
                     "train.zip",
                     "train",
                     "train_all",
                     "sampleSubmission.csv",
                     "small_train",
                     "small_val"
                     ])

for item in pre_existing_items:
    if item.exists():
        # We need to use different functions for files and directories.
        if item.is_dir():
            shutil.rmtree(item)
        elif item.is_file():
            item.unlink()
        else:
            print("Unknown item: {}, remove manually".format(item))


# Depending on your machine the following might take some seconds to run
# `!unzip` runs the unzip command on your system, i.e. you must have unzip installed.
# The `unzip` command is commonly installed but if not,
# the command might fail and you need to unzip manually.
!unzip -q dogs-vs-cats.zip
!unzip -q test1.zip
!unzip -q train.zip

Path("test1").rename("test")
Path("train").rename("train_all")


# Remove sub zip filess
Path("test1.zip").unlink()
Path("train.zip").unlink()

# Take a look at your current directory. Apart from notebook files (those ending in *.ipynb) you should see
# dogs-vs-cats.zip
# sampleSubmission.csv
# test
# train_all

Now we'll examine the data inside the directory `train_all`.
It contains files like this:

```
<id>.jpg
<id>.jpg
```
where each id is on the from `<label>.<number>`, e.g. `cat.123`.

Inside the directory `test` are unlabelled images

```
<id>.jpg
```
where each id is a single number.

Predictions on these unknown images is what you would submit to participate in the contest.

Let's count them separately.

In [ ]:

train_all_path = Path.cwd() / "train_all"

# Get a list of all filenames inside (these will be used for training and validation)
# The asterisk '*' is a so called wildcard, i.e. we tell the `glob` method to find any cat/dog images,
# regardless of their id.
all_cat_filenames = list(train_all_path.glob("cat.*.jpg"))
all_dog_filenames = list(train_all_path.glob("dog.*.jpg"))

test_path = Path.cwd() / "test"
all_test_filenames = list(test_path.glob("*.jpg"))
print(f"Found {len(all_cat_filenames)} images of cats.\nFound {len(all_dog_filenames)} images of dogs.\nFound {len(all_test_filenames)} test images")

We'll create `'small_train'` and `'small_val'` folders for a smaller subset of the original dataset (the assignment asks for 20%).

In [ ]:
# Get a subset of the entire training dataset (20%)
_, few_cat_filenames, _, few_dog_filenames = train_test_split(all_cat_filenames, 
                                                              all_dog_filenames, 
                                                              test_size=0.2, random_state=1)

In [ ]:
# Split it into training and validation sets
split_ratio_small_dataset = 0.3

few_cat_filenames_train, few_cat_filenames_val, few_dog_filenames_train, few_dog_filenames_val = \
train_test_split(few_cat_filenames, 
              few_dog_filenames, 
              test_size = split_ratio_small_dataset,
              random_state = 2)

print("The smaller dataset will be comprised of:")
print(f"Training:\t{len(few_cat_filenames_train)} cats and {len(few_dog_filenames_train)} dogs")
print(f"Validation:\t{len(few_cat_filenames_val)} cats and {len(few_dog_filenames_val)} dogs")

In [ ]:
# Create the train and val directories and subdirectories
subdirectories = {"small_train/cats": few_cat_filenames_train,
                  "small_train/dogs": few_dog_filenames_train,
                  "small_val/cats": few_cat_filenames_val,
                  "small_val/dogs": few_dog_filenames_val
                 }

for subdirectory in subdirectories.keys():
    subdirectory = Path(subdirectory)
    subdirectory.mkdir(parents=True, exist_ok=True)

# Put the training and validation data in the respective folders
def fill_sub_dir(sub_dir, file_subset):
    """This function copies files from the `train_all` to a `<sub_dir>`
    A more efficient solution would be to use "symbolic links" (see https://kb.iu.edu/d/abbe)
    but for simplicity hard copies is used instead.
    """
    for file in file_subset:
        file_path = Path.cwd() / sub_dir / file.name
        shutil.copyfile(file, file_path)
        
for sub_dir, file_subset in subdirectories.items():
    fill_sub_dir(sub_dir, file_subset)

Now we create the `val` and `train` folders for the entire dataset. You need to specify the train/val split (to something reasonable).

In [ ]:
# Choose
split_ratio_big_dataset = 0.06

if split_ratio_big_dataset is None:
    raise ValueError("`split_ratio_big_dataset` must have a value between 0 and 1.")

# Split it
all_cat_filenames_train, all_cat_filenames_val, all_dog_filenames_train, all_dog_filenames_val = \
train_test_split(all_cat_filenames,
                 all_dog_filenames,
                 test_size=split_ratio_big_dataset,
                 random_state=3)

print("The full dataset will be comprised of:")
print(f"Train:\t{len(all_cat_filenames_train)} cats and {len(all_dog_filenames_train)} dogs.")
print(f"Val:\t{len(all_cat_filenames_val)} cats and {len(all_dog_filenames_val)} dogs")

In [ ]:
# Create the train and val directories and subdirectories
subdirectories = {"train/cats": all_cat_filenames_train,
                 "train/dogs": all_dog_filenames_train,
                 "val/cats": all_cat_filenames_val,
                 "val/dogs": all_dog_filenames_val
                 }

for subdirectory in subdirectories.keys():
    subdirectory = Path(subdirectory)
    subdirectory.mkdir(parents=True, exist_ok=True)

for sub_dir, file_subset in subdirectories.items():
    fill_sub_dir(sub_dir, file_subset)